In [1]:
%load_ext cython

In [2]:
%load_ext line_profiler

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import scale

In [4]:
df = pd.read_feather('../train.feather')

/usr/local/anaconda3/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning: .labels was deprecated in version 0.24.0. Use .codes instead.
  labels, = index.labels


In [27]:
X = df.iloc[:, 2:].values.astype(np.float32)

In [28]:
%%cython -a --compile-args=-Xpreprocessor --compile-args=-fopenmp --link-args=-lomp
import numpy as np
cimport numpy as np
cimport cython
from libc.math cimport sqrt, acos, pi
from cython.parallel import prange

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def polar(float[:, ::1] A):
    """Convert cartesian to polar coordinates."""
    cdef int m = A.shape[0]
    cdef int n = A.shape[1]
    
    cdef float r_tmp, denom, tmp, x_tmp1, x_tmp2
    cdef float[:, ::1] view = A
    cdef np.ndarray[np.float32_t, ndim=2] ret = np.empty((m, n), dtype=np.float32)
    cdef float[:, ::1] ret_view = ret
    
    with nogil:
        for i in range(m):
            x_tmp1 = view[i, n - 1]
            x_tmp2 = view[i, n - 2]
            r_tmp = x_tmp1 * x_tmp1 + x_tmp2 * x_tmp2
            tmp = view[i, 0]
            if tmp >= 0:
                ret_view[i, n - 1] = acos(tmp / sqrt(r_tmp))
            else:
                ret_view[i, n - 1] = (2 * pi) - acos(tmp / sqrt(r_tmp))

            for j in range(n - 2, 0, -1):
                x_tmp1 = view[i, j]
                r_tmp = r_tmp + x_tmp1 * view[i, j]
                ret_view[i, j] = acos(view[i, j] / sqrt(r_tmp))

            ret_view[i, 0] = sqrt(r_tmp)  
    np.nan_to_num(ret, copy=False)
    return ret

@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
def cy_distance_subset(float[:, ::1] A, int start, int end):
    cdef int m = A.shape[0]
    cdef int n = A.shape[1]
    cdef int cols = end - start
    
    assert start <= end
    assert start >= 0
    assert end <= m
    cdef np.ndarray[np.float32_t, ndim=2] ret = np.empty((m, cols), dtype=np.float32)
    cdef float[:, ::1] data = ret
    cdef int i, j, k
    cdef float val, denom1, denom2, numer, a, b
    cdef int _start = start
    with nogil:        
        for i in prange(m):
            for j in range(cols):
                if i == j:
                    val = 1
                else:
                    # Cosine similarity
                    numer = 0
                    denom1 = 0
                    denom2 = 0
                    for k in range(n):
                        a = A[i, k]
                        b = A[start + j, k]
                        numer = numer + a * b
                        denom1 = denom1 + a * a
                        denom2 = denom2 + b * b
                    val = numer / (sqrt(denom1) * sqrt(denom2))
                    
                data[i, j] = val
    return ret

In [29]:
x = np.array([[0, 0], [4, 5]], dtype=np.float32)
polar(x)

array([[0.       , 0.       ],
       [6.4031243, 0.8960554]], dtype=float32)

In [30]:
import cmath
t1 = complex(0, 0)
t2 = complex(4, 5)
cmath.polar(t1), cmath.polar(t2)

((0.0, 0.0), (6.4031242374328485, 0.8960553845713439))

In [31]:
import numpy as np

from sklearn.utils.extmath import randomized_svd
from scipy.linalg import qr


def fullsvd(A, k):
    u, s, v = np.linalg.svd(A, full_matrices=False)
    return u[:, :k], s[:k], v[:, :k]

def merge(us1, us2, k):
    u1, s1 = us1
    u2, s2 = us2
    
    k1 = s1.shape[0]
    k2 = s2.shape[0]
    
    Z = u1.T @ u2
    u_, r = qr(u2 - u1 @ Z, mode='economic')
    X = np.zeros((k1 + k2, k1 + k2))
    X[:k1, :k1] = np.diag(s1)
    X[:k1, k1:] = Z @ np.diag(s2)
    X[k1:, k1:] = r @ np.diag(s2)
    
    ur, sr, _ = fullsvd(X, k)
    r1 = ur[:k1, :]
    r2 = ur[k2:, :]
    urr = (u1 @ r1 + u_ @ r2)[:, :k]
    return urr, sr

def tsvd_dm(A, k=20, splits=2, randomized=False, split_k=None):
    if split_k is None:
        split_k = k
        
    m, n = A.shape
    split_size = m // splits
    results = []
    
    for i in range(splits):
        A_sub = cy_distance_subset(A, i * split_size, (i + 1) * split_size)
        print(A_sub.shape, split_size)
        print(f'Subset {i} generated')
        if randomized:
            *us, _ = randomized_svd(A_sub, split_k)
        else:
            *us, _ = fullsvd(A_sub, split_k)
        print(f'TSVD completed')
        results.append(us)
    
    if len(results) % 2 == 1:
        results.extend([None])
    
    while len(results) > 1:
        work_stack = []
        pairs = zip(results[::2], results[1::2])
        for us1, us2 in pairs:
            if us2 is None:
                work_stack.append(us1)
            else:
                merged = merge(us1, us2, k=k)
                work_stack.append(merged)
                
        if len(results) > 1 and len(results) % 2 == 1:
            work_stack.extend([None])
        results = work_stack
        
    return work_stack[0]

In [32]:
scaledX = polar(np.ascontiguousarray(scale(X)))

/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:176: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
/usr/local/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:193: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [33]:
%%time
u_, s_ = tsvd_dm(scaledX, k=256, splits=32, randomized=True)

(200000, 6250) 6250
Subset 0 generated
TSVD completed
(200000, 6250) 6250
Subset 1 generated
TSVD completed
(200000, 6250) 6250
Subset 2 generated
TSVD completed
(200000, 6250) 6250
Subset 3 generated
TSVD completed
(200000, 6250) 6250
Subset 4 generated
TSVD completed
(200000, 6250) 6250
Subset 5 generated
TSVD completed
(200000, 6250) 6250
Subset 6 generated
TSVD completed
(200000, 6250) 6250
Subset 7 generated
TSVD completed
(200000, 6250) 6250
Subset 8 generated
TSVD completed
(200000, 6250) 6250
Subset 9 generated
TSVD completed
(200000, 6250) 6250
Subset 10 generated
TSVD completed
(200000, 6250) 6250
Subset 11 generated
TSVD completed
(200000, 6250) 6250
Subset 12 generated
TSVD completed
(200000, 6250) 6250
Subset 13 generated
TSVD completed
(200000, 6250) 6250
Subset 14 generated
TSVD completed
(200000, 6250) 6250
Subset 15 generated
TSVD completed
(200000, 6250) 6250
Subset 16 generated
TSVD completed
(200000, 6250) 6250
Subset 17 generated
TSVD completed
(200000, 6250) 6250


In [34]:
u_

array([[ 2.23329996e-03, -4.50305762e-05, -1.63995031e-05, ...,
         9.23384342e-03, -7.32435296e-03, -2.06722987e-02],
       [ 2.23904303e-03, -4.37589266e-05, -1.50274995e-05, ...,
        -1.42078221e-03,  2.90455183e-03, -2.20233713e-03],
       [ 2.22991245e-03, -4.34418987e-05, -1.43318189e-05, ...,
         9.84443870e-03,  1.92597491e-02, -2.83856845e-02],
       ...,
       [ 2.23981194e-03, -4.38488438e-05, -1.45968639e-05, ...,
        -2.50437212e-04, -3.68999544e-04,  9.19217300e-05],
       [ 2.23113970e-03, -4.42691674e-05, -1.53046902e-05, ...,
        -4.86012248e-04, -1.35428186e-03, -1.86371828e-04],
       [ 2.23968731e-03, -4.39190640e-05, -1.47362449e-05, ...,
        -1.07455434e-03,  8.36278454e-04, -7.07521603e-04]])

In [38]:
test = np.random.random((1000, 5)).astype(np.float32)

In [39]:
np.set_printoptions(suppress=False, precision=5)
tsvd_dm(test, k=20, split_k=100, splits=8, randomized=True)

(1000, 125) 125
Subset 0 generated
TSVD completed
(1000, 125) 125
Subset 1 generated
TSVD completed
(1000, 125) 125
Subset 2 generated
TSVD completed
(1000, 125) 125
Subset 3 generated
TSVD completed
(1000, 125) 125
Subset 4 generated
TSVD completed
(1000, 125) 125
Subset 5 generated
TSVD completed
(1000, 125) 125
Subset 6 generated
TSVD completed
(1000, 125) 125
Subset 7 generated
TSVD completed


(array([[ 0.03317, -0.03126, -0.02837, ..., -0.00103,  0.01143,  0.01308],
        [ 0.03459,  0.00192,  0.01074, ...,  0.00211,  0.00297,  0.00072],
        [ 0.02829,  0.04725, -0.05226, ..., -0.00672, -0.00184, -0.02342],
        ...,
        [ 0.03288, -0.01295, -0.00853, ..., -0.00197, -0.00193, -0.0004 ],
        [ 0.0343 , -0.03534, -0.00767, ..., -0.00128,  0.00025, -0.00113],
        [ 0.02793, -0.00138,  0.07975, ...,  0.00331,  0.00012,  0.00473]]),
 array([7.78096e+02, 5.91851e+01, 5.67800e+01, 5.49997e+01, 5.09420e+01,
        1.25378e+00, 1.08586e+00, 9.88866e-01, 8.50822e-01, 8.33857e-01,
        8.03733e-01, 7.68136e-01, 7.56264e-01, 7.52898e-01, 7.45135e-01,
        7.34743e-01, 7.32305e-01, 7.27463e-01, 7.14419e-01, 6.97193e-01]))

In [40]:
u, s, v = fullsvd(cy_distance_subset(test, 0, 1000), 20)
u, s

(array([[-0.03313,  0.03175, -0.02861, ...,  0.03129,  0.01088,  0.02663],
        [-0.03457, -0.00177,  0.01075, ...,  0.00825,  0.00216, -0.04409],
        [-0.0282 , -0.0474 , -0.0529 , ...,  0.00046, -0.01445,  0.03134],
        ...,
        [-0.03289,  0.01285, -0.00872, ...,  0.02405, -0.02098, -0.00832],
        [-0.03431,  0.03528, -0.00776, ..., -0.00929, -0.01536,  0.0145 ],
        [-0.02793,  0.00163,  0.07973, ..., -0.02279,  0.03107, -0.03795]],
       dtype=float32),
 array([7.77901e+02, 5.92396e+01, 5.68290e+01, 5.50381e+01, 5.09926e+01,
        2.40677e-06, 2.38656e-06, 2.36640e-06, 2.36152e-06, 2.34998e-06,
        2.33300e-06, 2.33038e-06, 2.31951e-06, 2.30577e-06, 2.30026e-06,
        2.29809e-06, 2.29331e-06, 2.28218e-06, 2.27821e-06, 2.27747e-06],
       dtype=float32))

In [36]:
cy_distance_subset(test[:7], 0, 7)

array([[1.     , 0.63343, 0.52882, 0.8548 , 0.85561, 0.50162, 0.73045],
       [0.63343, 1.     , 0.88059, 0.83249, 0.88845, 0.77294, 0.94409],
       [0.52882, 0.88059, 1.     , 0.83658, 0.87299, 0.70342, 0.88753],
       [0.8548 , 0.83249, 0.83658, 1.     , 0.94649, 0.57287, 0.90613],
       [0.85561, 0.88845, 0.87299, 0.94649, 1.     , 0.78465, 0.9623 ],
       [0.50162, 0.77294, 0.70342, 0.57287, 0.78465, 1.     , 0.84579],
       [0.73045, 0.94409, 0.88753, 0.90613, 0.9623 , 0.84579, 1.     ]])

In [37]:
cy_distance_subset(test, 0, 3)[:7]

array([[1.     , 0.63343, 0.52882],
       [0.63343, 1.     , 0.88059],
       [0.52882, 0.88059, 1.     ],
       [0.8548 , 0.83249, 0.83658],
       [0.85561, 0.88845, 0.87299],
       [0.50162, 0.77294, 0.70342],
       [0.73045, 0.94409, 0.88753]])